# This is Nir Levy and Shahar Michel Project, about predicting the NBA Draft picks. In this section, we did the crawling in basketball.realgm.com

In [1]:
import numpy as np
import json
import os
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [2]:
# Here, we built the 10 first lists that will collect the first data about the players.
Web_url = "https://basketball.realgm.com"
Name = list()
Position = list()
Pick = list()
Team = list()
Height = list()
Weight = list()
Year = list()
Nationality = list()
Age = list()
linksToPastPlayersPages = list()
# We ran in a loop between the years 2003 to 2021 and took links to each player's page
# We took their name, position, the pick that they were picked or 0 if they did not drafted, the team that they played before,
# their height and weight, their nationality, the age that they get drafted and the selection year.
for i in range(2003,2022):
    Base_url = "https://basketball.realgm.com/nba/draft/past_drafts/"+str(i)
    Base_html = requests.get(Base_url)
    Base_soup = BeautifulSoup(Base_html.content,'html.parser')
    for j in range(0,2):
        Past_Draft_Table= Base_soup("table", attrs={"class":"tablesaw"})[j]
        for rows in Past_Draft_Table("tr"):
            Past_Players_Cells = rows("td")
            if(len(Past_Players_Cells)<12):
                continue
            Name.append(Past_Players_Cells[1].get_text())
            Position.append(Past_Players_Cells[4].get_text())
            Pick.append(Past_Players_Cells[0].get_text())
            Team.append(Past_Players_Cells[9].get_text())
            if Past_Players_Cells[11].get_text() == 'N/A' or Past_Players_Cells[11].get_text() == '-':
                Nationality.append("N/A")
            else:
                Nationality.append(Past_Players_Cells[11].get_text())
            if Past_Players_Cells[7].get_text() == 'N/A' or Past_Players_Cells[7].get_text() == '-':
                Age.append("N/A")
            else:
                Age.append(Past_Players_Cells[7].get_text())
            if Past_Players_Cells[5].get_text() == 'N/A':
                Height.append("N/A")
            else:
                Height.append(int((float(Past_Players_Cells[5].get_text()[0]) * 12)+float(Past_Players_Cells[5].get_text()[2:]))*2.54)
            if Past_Players_Cells[6].get_text() == 'N/A':
                Weight.append("N/A")
            else:
                Weight.append(int(int(Past_Players_Cells[6].get_text()) * 0.453592))
            linksToPastPlayersPages+=[t['href'] for t in Past_Players_Cells[1]]
            Year.append(i)
    Past_Draft_Table= Base_soup("table", attrs={"class":"tablesaw"})[2]
    for rows in Past_Draft_Table("tr"):
        Past_Players_Cells = rows("td")
        if(len(Past_Players_Cells)<9):
            continue
        Name.append(Past_Players_Cells[0].get_text())
        Position.append(Past_Players_Cells[1].get_text())
        Pick.append(0)
        Team.append(Past_Players_Cells[6].get_text())
        if Past_Players_Cells[8].get_text() == 'N/A' or Past_Players_Cells[8].get_text() == '-':
            Nationality.append("N/A")
        else:
            Nationality.append(Past_Players_Cells[8].get_text())
        if Past_Players_Cells[4].get_text() == 'N/A' or Past_Players_Cells[4].get_text() == '-':
            Age.append("N/A")
        else:
            Age.append(Past_Players_Cells[4].get_text())
        if Past_Players_Cells[2].get_text() == 'N/A' or Past_Players_Cells[2].get_text() == 'N/A':
            Height.append("N/A")
        else:
            Height.append(int((float(Past_Players_Cells[2].get_text()[0]) * 12)+float(Past_Players_Cells[2].get_text()[2:]))*2.54)
        if Past_Players_Cells[3].get_text() == 'N/A' or Past_Players_Cells[3].get_text() == '-':
            Weight.append("N/A")
        else:
            Weight.append(int(int(Past_Players_Cells[3].get_text()) * 0.453592))
        linksToPastPlayersPages+=[t['href'] for t in Past_Players_Cells[0]]
        Year.append(i)
for i in range(len(linksToPastPlayersPages)):
    linksToPastPlayersPages[i] = Web_url+linksToPastPlayersPages[i]

In [3]:
df = pd.DataFrame({"name":Name,"Age":Age, "Nationality":Nationality, "Pick":Pick,"Team":Team,"Selection year":Year, "Position":Position, "Height":Height, "Weight":Weight})
print(df)

                   name Age    Nationality Pick  \
0          LeBron James  18  United States    1   
1         Darko Milicic  18         Serbia    2   
2       Carmelo Anthony  19  United States    3   
3            Chris Bosh  19  United States    4   
4           Dwyane Wade  21  United States    5   
...                 ...  ..            ...  ...   
2297          Dru Smith  23  United States    0   
2298  D.J. Stewart, Jr.  22  United States    0   
2299      Anthony Tarke  24  United States    0   
2300       MaCio Teague  24  United States    0   
2301         Ibi Watson  23  United States    0   

                                         Team  Selection year Position  \
0     St. Vincent St. Mary High School (Ohio)            2003        F   
1                 KK Vrsac Swisslion (Serbia)            2003       FC   
2                                    Syracuse            2003        F   
3                                Georgia Tech            2003       PF   
4                

####  GP- Game played, PTS- points, MIN- minutes, FGM- Field goals made, FGA- Field goals attempted, FGP- Field goals percentage
#### TPM- Three points made, TPA- Three points attempted, TPP- Three points percentage, FTM- Free throws made
#### FPA- Free throws attempted, FTP- Free throws percentage, OFFR- offensive rebound, DEFR- defensive rebound, STL- steals
#### TRB-  total rebounds, AST- assissts, BLK- blocks, TOV- turnovers, NIG- NCCA, Internacional or G-league 

In [4]:
# Here we entered to each player's page and took the stats from his last season in his team before he signed to the draft
# We took the totals  of each stat, and divided in the GP that every player playd.
n = -1
GP = list()
PTS = list()
MIN = list()
FGM = list()
FGA = list()
FGP = list()
TPM = list()
TPA = list()
TPP = list()
FTM = list()
FTA = list()
FTP = list()
OFFR = list()
DEFR = list()
TRB = list()
AST = list()
STL = list()
BLK = list()
TOV = list()
NIG = list()
f = 0
r = 0
z = 0

for PlayerLink in linksToPastPlayersPages:
    Player_url = PlayerLink
    Player_html = requests.get(Player_url)
    Player_soup = BeautifulSoup(Player_html.content,'html.parser')
    f = 0
    r = 0
    z = 0
    n = n+1
    got_det = 0
    for category in Player_soup.findAll("span", {"class":'nowrap'}):
        if category.find("a", {"href":'#NCAA'}) != None:
            if Player_soup.select_one('h2:-soup-contains("NCAA Season Stats - Totals")') == None:
                continue
            NCAATable = Player_soup.select_one('h2:-soup-contains("NCAA Season Stats - Totals")').find_next('table')
            first_row = NCAATable("tr")[0]
            for row in NCAATable("tr"):
                if row == first_row:
                    continue
                Cells = row("td")
                if Cells[0].get_text()[:4] == str(int(Year[n])-1) and f!=1:
                    NIG.append("NCAA")
                    got_det = 1
                    f = 1
                    if(row("td")[3].get_text() == '-'):
                        GP.append('N/A')
                        MIN.append('N/A')
                        PTS.append('N/A')
                        FGA.append('N/A')
                        FGM.append('N/A')
                        FGP.append('N/A')
                        TPA.append('N/A')
                        TPM.append('N/A')
                        TPP.append('N/A')
                        FTA.append('N/A')
                        FTM.append('N/A')
                        FTP.append('N/A')
                        OFFR.append('N/A')
                        DEFR.append('N/A')
                        TRB.append('N/A')
                        AST.append('N/A')
                        STL.append('N/A')
                        BLK.append('N/A')
                        TOV.append('N/A')
                    else:
                        GP.append(row("td")[3].get_text())
                        if row("td")[5].get_text() == 'N/A' or row("td")[5].get_text() == '-':
                            MIN.append(0)
                        else:
                            MIN.append((float(row("td")[5].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[23].get_text() == 'N/A' or row("td")[23].get_text() == '-':
                            PTS.append(0)
                        else:
                            PTS.append((float(row("td")[23].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[7].get_text() == 'N/A' or row("td")[7].get_text() == '-':
                            FGA.append(0)
                            FGM.append(0)
                            FGP.append(0)
                        else:
                            FGA.append((float(row("td")[7].get_text()))/(float(row("td")[3].get_text())))
                            FGM.append((float(row("td")[6].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[7].get_text == "0"  or row("td")[6].get_text() == "0"):
                                FGP.append(0)
                            else:
                                FGP.append((float(row("td")[6].get_text()))/(float(row("td")[7].get_text())))

                        if row("td")[10].get_text() == 'N/A' or row("td")[10].get_text() == '-':
                            TPA.append(0)
                            TPM.append(0)
                            TPP.append(0)
                        else:
                            TPA.append((float(row("td")[10].get_text()))/(float(row("td")[3].get_text())))
                            TPM.append((float(row("td")[9].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[10].get_text == "0" or row("td")[9].get_text() == "0"):
                                TPP.append(0)
                            else:
                                TPP.append((float(row("td")[9].get_text()))/(float(row("td")[10].get_text())))

                        if row("td")[13].get_text() == 'N/A' or row("td")[13].get_text() == '-':
                            FTA.append(0)
                            FTM.append(0)
                            FTP.append(0)
                        else:
                            FTA.append((float(row("td")[13].get_text()))/(float(row("td")[3].get_text())))
                            FTM.append((float(row("td")[12].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[13].get_text == "0"  or row("td")[12].get_text() == "0"):
                                FTP.append(0)
                            else:
                                FTP.append((float(row("td")[12].get_text()))/(float(row("td")[13].get_text())))

                        if row("td")[15].get_text() == 'N/A' or row("td")[15].get_text() == '-':
                            OFFR.append(0)
                        else:
                            OFFR.append((float(row("td")[15].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[16].get_text() == 'N/A' or row("td")[16].get_text() == '-':
                            DEFR.append(0)
                        else:
                            DEFR.append((float(row("td")[16].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[17].get_text() == 'N/A' or row("td")[17].get_text() == '-':
                            TRB.append(0)
                        else:
                            TRB.append((float(row("td")[17].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[18].get_text() == 'N/A' or row("td")[18].get_text() == '-':
                            AST.append(0)
                        else:
                            AST.append((float(row("td")[18].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[19].get_text() == 'N/A' or row("td")[19].get_text() == '-':
                            STL.append(0)
                        else:
                            STL.append((float(row("td")[19].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[20].get_text() == 'N/A' or row("td")[20].get_text() == '-':
                            BLK.append(0)
                        else:
                            BLK.append((float(row("td")[20].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[22].get_text() == 'N/A' or row("td")[22].get_text() == '-':
                            TOV.append(0)
                        else:
                            TOV.append((float(row("td")[22].get_text()))/(float(row("td")[3].get_text())))

        elif category.find("a", {"href":'#International'})!= None and f!=1:
            if Player_soup.select_one('h2:-soup-contains("International Regular Season Stats - Totals")') == None:
                continue
            INTable = Player_soup.select_one('h2:-soup-contains("International Regular Season Stats - Totals")').find_next('table')
            first_row = INTable("tr")[0]
            for row in INTable("tr"):
                if row == first_row:
                    continue
                Cells = row("td")
                if Cells[0].get_text()[:4] == str(int(Year[n])-1) and r!=1:
                    NIG.append("International")
                    got_det = 1
                    r = 1
                    if (row("td")[3].get_text() == '-'):
                            GP.append('N/A')
                            MIN.append('N/A')
                            PTS.append('N/A')
                            FGA.append('N/A')
                            FGM.append('N/A')
                            FGP.append('N/A')
                            TPA.append('N/A')
                            TPM.append('N/A')
                            TPP.append('N/A')
                            FTA.append('N/A')
                            FTM.append('N/A')
                            FTP.append('N/A')
                            OFFR.append('N/A')
                            DEFR.append('N/A')
                            TRB.append('N/A')
                            AST.append('N/A')
                            STL.append('N/A')
                            BLK.append('N/A')
                            TOV.append('N/A')
                    else:
                        GP.append(row("td")[3].get_text())
                        if row("td")[5].get_text() == 'N/A' or row("td")[5].get_text() == '-':
                            MIN.append(0)
                        else:
                            MIN.append((float(row("td")[5].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[23].get_text() == 'N/A' or row("td")[23].get_text() == '-':
                            PTS.append(0)
                        else:
                            PTS.append((float(row("td")[23].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[7].get_text() == 'N/A' or row("td")[7].get_text() == '-':
                            FGA.append(0)
                            FGM.append(0)
                            FGP.append(0)
                        else:
                            FGA.append((float(row("td")[7].get_text()))/(float(row("td")[3].get_text())))
                            FGM.append((float(row("td")[6].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[7].get_text == "0"  or row("td")[6].get_text() == "0"):
                                FGP.append(0)
                            else:
                                FGP.append((float(row("td")[6].get_text()))/(float(row("td")[7].get_text())))

                        if row("td")[10].get_text() == 'N/A' or row("td")[10].get_text() == '-':
                            TPA.append(0)
                            TPM.append(0)
                            TPP.append(0)
                        else:
                            TPA.append((float(row("td")[10].get_text()))/(float(row("td")[3].get_text())))
                            TPM.append((float(row("td")[9].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[10].get_text == "0" or row("td")[9].get_text() == "0"):
                                TPP.append(0)
                            else:
                                TPP.append((float(row("td")[9].get_text()))/(float(row("td")[10].get_text())))

                        if row("td")[13].get_text() == 'N/A' or row("td")[13].get_text() == '-':
                            FTA.append(0)
                            FTM.append(0)
                            FTP.append(0)
                        else:
                            FTA.append((float(row("td")[13].get_text()))/(float(row("td")[3].get_text())))
                            FTM.append((float(row("td")[12].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[13].get_text == "0"  or row("td")[12].get_text() == "0"):
                                FTP.append(0)
                            else:
                                FTP.append((float(row("td")[12].get_text()))/(float(row("td")[13].get_text())))

                        if row("td")[15].get_text() == 'N/A' or row("td")[15].get_text() == '-':
                            OFFR.append(0)
                        else:
                            OFFR.append((float(row("td")[15].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[16].get_text() == 'N/A' or row("td")[16].get_text() == '-':
                            DEFR.append(0)
                        else:
                            DEFR.append((float(row("td")[16].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[17].get_text() == 'N/A' or row("td")[17].get_text() == '-':
                            TRB.append(0)
                        else:
                            TRB.append((float(row("td")[17].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[18].get_text() == 'N/A' or row("td")[18].get_text() == '-':
                            AST.append(0)
                        else:
                            AST.append((float(row("td")[18].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[19].get_text() == 'N/A' or row("td")[19].get_text() == '-':
                            STL.append(0)
                        else:
                            STL.append((float(row("td")[19].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[20].get_text() == 'N/A' or row("td")[20].get_text() == '-':
                            BLK.append(0)
                        else:
                            BLK.append((float(row("td")[20].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[22].get_text() == 'N/A' or row("td")[22].get_text() == '-':
                            TOV.append(0)
                        else:
                            TOV.append((float(row("td")[22].get_text()))/(float(row("td")[3].get_text())))

        elif category.find("a", {"href":'#D-League'}) != None and f!=1 and r!=1:
            if Player_soup.select_one('h2:-soup-contains("G League Regular Season Stats - Totals")')== None:
                continue
            GL_Table = Player_soup.select_one('h2:-soup-contains("G League Regular Season Stats - Totals")').find_next('table')
            first_row = GL_Table("tr")[0]
            for row in GL_Table("tr"):
                if row == first_row:
                    continue
                Cells = row("td")       
                if(len(Cells)<24):
                    continue
                if Cells[0].get_text()[:4] == str(int(Year[n])-1) and z!=1:
                    NIG.append("G League")
                    got_det = 1
                    z = 1
                    if (row("td")[3].get_text() == '-'):
                        GP.append('N/A')
                        MIN.append('N/A')
                        PTS.append('N/A')
                        FGA.append('N/A')
                        FGM.append('N/A')
                        FGP.append('N/A')
                        TPA.append('N/A')
                        TPM.append('N/A')
                        TPP.append('N/A')
                        FTA.append('N/A')
                        FTM.append('N/A')
                        FTP.append('N/A')
                        OFFR.append('N/A')
                        DEFR.append('N/A')
                        TRB.append('N/A')
                        AST.append('N/A')
                        STL.append('N/A')
                        BLK.append('N/A')
                        TOV.append('N/A')
                    else:
                        GP.append(row("td")[3].get_text())
                        if row("td")[5].get_text() == 'N/A' or row("td")[5].get_text() == '-':
                            MIN.append(0)
                        else:
                            MIN.append((float(row("td")[5].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[23].get_text() == 'N/A' or row("td")[23].get_text() == '-':
                            PTS.append(0)
                        else:
                            PTS.append((float(row("td")[23].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[7].get_text() == 'N/A' or row("td")[7].get_text() == '-':
                            FGA.append(0)
                            FGM.append(0)
                            FGP.append(0)
                        else:
                            FGA.append((float(row("td")[7].get_text()))/(float(row("td")[3].get_text())))
                            FGM.append((float(row("td")[6].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[7].get_text == "0"  or row("td")[6].get_text() == "0"):
                                FGP.append(0)
                            else:
                                FGP.append((float(row("td")[6].get_text()))/(float(row("td")[7].get_text())))

                        if row("td")[10].get_text() == 'N/A' or row("td")[10].get_text() == '-':
                            TPA.append(0)
                            TPM.append(0)
                            TPP.append(0)
                        else:
                            TPA.append((float(row("td")[10].get_text()))/(float(row("td")[3].get_text())))
                            TPM.append((float(row("td")[9].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[10].get_text == "0" or row("td")[9].get_text() == "0"):
                                TPP.append(0)
                            else:
                                TPP.append((float(row("td")[9].get_text()))/(float(row("td")[10].get_text())))

                        if row("td")[13].get_text() == 'N/A' or row("td")[13].get_text() == '-':
                            FTA.append(0)
                            FTM.append(0)
                            FTP.append(0)
                        else:
                            FTA.append((float(row("td")[13].get_text()))/(float(row("td")[3].get_text())))
                            FTM.append((float(row("td")[12].get_text()))/(float(row("td")[3].get_text())))
                            if(row("td")[13].get_text == "0"  or row("td")[12].get_text() == "0"):
                                FTP.append(0)
                            else:
                                FTP.append((float(row("td")[12].get_text()))/(float(row("td")[13].get_text())))

                        if row("td")[15].get_text() == 'N/A' or row("td")[15].get_text() == '-':
                            OFFR.append(0)
                        else:
                            OFFR.append((float(row("td")[15].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[16].get_text() == 'N/A' or row("td")[16].get_text() == '-':
                            DEFR.append(0)
                        else:
                            DEFR.append((float(row("td")[16].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[17].get_text() == 'N/A' or row("td")[17].get_text() == '-':
                            TRB.append(0)
                        else:
                            TRB.append((float(row("td")[17].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[18].get_text() == 'N/A' or row("td")[18].get_text() == '-':
                            AST.append(0)
                        else:
                            AST.append((float(row("td")[18].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[19].get_text() == 'N/A' or row("td")[19].get_text() == '-':
                            STL.append(0)
                        else:
                            STL.append((float(row("td")[19].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[20].get_text() == 'N/A' or row("td")[20].get_text() == '-':
                            BLK.append(0)
                        else:
                            BLK.append((float(row("td")[20].get_text()))/(float(row("td")[3].get_text())))

                        if row("td")[22].get_text() == 'N/A' or row("td")[22].get_text() == '-':
                                TOV.append(0)
                        else:
                            TOV.append((float(row("td")[22].get_text()))/(float(row("td")[3].get_text())))
    if got_det == 0:
        NIG.append('N/A')
        GP.append('N/A')
        MIN.append('N/A')
        PTS.append('N/A')
        FGA.append('N/A')
        FGM.append('N/A')
        FGP.append('N/A')
        TPA.append('N/A')
        TPM.append('N/A')
        TPP.append('N/A')
        FTA.append('N/A')
        FTM.append('N/A')
        FTP.append('N/A')
        OFFR.append('N/A')
        DEFR.append('N/A')
        TRB.append('N/A')                     
        AST.append('N/A')
        STL.append('N/A')
        BLK.append('N/A')
        TOV.append('N/A')
    print(AST[n], Name[n], NIG[n], Year[n])

N/A LeBron James N/A 2003
N/A Darko Milicic N/A 2003
2.2 Carmelo Anthony NCAA 2003
1.2258064516129032 Chris Bosh NCAA 2003
4.393939393939394 Dwyane Wade NCAA 2003
1.2580645161290323 Chris Kaman NCAA 2003
3.5135135135135136 Kirk Hinrich NCAA 2003
7.696969696969697 T.J. Ford NCAA 2003
1.9411764705882353 Mike Sweetney NCAA 2003
2.0 Jarvis Hayes NCAA 2003
1.6153846153846154 Mickael Pietrus International 2003
2.210526315789474 Nick Collison NCAA 2003
5.46875 Marcus Banks NCAA 2003
6.606060606060606 Luke Ridnour NCAA 2003
4.96875 Reece Gaines NCAA 2003
3.7096774193548385 Troy Bell NCAA 2003
1.0769230769230769 Zarko Cabarkapa International 2003
3.1875 David West NCAA 2003
1.0 Sasha Pavlovic International 2003
0.8787878787878788 Dahntay Jones NCAA 2003
3.0 Boris Diaw International 2003
2.8333333333333335 Zoran Planinic International 2003
N/A Travis Outlaw N/A 2003
2.0 Brian Cook NCAA 2003
1.6875 Carlos Delfino International 2003
N/A Ndudi Ebi N/A 2003
N/A Kendrick Perkins N/A 2003
N/A Leandro 

3.8947368421052633 Francisco Garcia NCAA 2005
3.8461538461538463 Luther Head NCAA 2005
0.2222222222222222 Johan Petro International 2005
0.8484848484848485 Jason Maxiell NCAA 2005
1.606060606060606 Linas Kleiza NCAA 2005
0.12121212121212122 Ian Mahinmi International 2005
1.3846153846153846 Wayne Simien NCAA 2005
2.1875 David Lee NCAA 2005
2.1666666666666665 Salim Stoudamire NCAA 2005
4.0 Daniel Ewing NCAA 2005
0.8333333333333334 Brandon Bass NCAA 2005
N/A C.J. Miles N/A 2005
N/A Ricky Sanchez N/A 2005
0.0 Ersan Ilyasova International 2005
1.5483870967741935 Ronny Turiaf NCAA 2005
6.956521739130435 Travis Diener NCAA 2005
1.6551724137931034 Von Wafer NCAA 2005
N/A Monta Ellis N/A 2005
N/A Roko Ukic N/A 2005
0.8275862068965517 Chris Taft NCAA 2005
0.4444444444444444 Mile Ilic International 2005
0.26666666666666666 Martynas Andriuskevicius International 2005
N/A Lou Williams N/A 2005
N/A Erazem Lorbek NCAA 2005
2.6923076923076925 Bracey Wright NCAA 2005
1.0909090909090908 Mickael Gelabale

0.2727272727272727 Aaron Pettway NCAA 2006
1.2121212121212122 Kevin Pittsnogle NCAA 2006
0.78125 Antywane Robinson NCAA 2006
4.3076923076923075 Dawan Robinson NCAA 2006
1.8666666666666667 Marcus Slaughter NCAA 2006
5.533333333333333 Curtis Stinson NCAA 2006
0.4411764705882353 Doug Thomas NCAA 2006
1.0 Eric Williams NCAA 2006
2.84375 Charles Lee NCAA 2006
1.3928571428571428 Jared Newson NCAA 2006
0.65625 Greg Oden NCAA 2007
1.3142857142857143 Kevin Durant NCAA 2007
2.1578947368421053 Al Horford NCAA 2007
6.102564102564102 Mike Conley NCAA 2007
3.189189189189189 Jeff Green NCAA 2007
N/A Jianlian Yi N/A 2007
2.945945945945946 Corey Brewer NCAA 2007
1.0 Brandan Wright NCAA 2007
2.25 Joakim Noah NCAA 2007
1.935483870967742 Spencer Hawes NCAA 2007
4.970588235294118 Acie Law NCAA 2007
2.032258064516129 Thaddeus Young NCAA 2007
2.1578947368421053 Julian Wright NCAA 2007
0.6857142857142857 Al Thornton NCAA 2007
5.517241379310345 Rodney Stuckey NCAA 2007
1.3513513513513513 Nick Young NCAA 2007
1

4.027777777777778 Drew Neitzel NCAA 2008
2.0 David Padgett NCAA 2008
0.7941176470588235 Shaun Pruitt NCAA 2008
0.9354838709677419 Charles Rhodes NCAA 2008
2.727272727272727 Maureece Rice NCAA 2008
8.13888888888889 Jason Richards NCAA 2008
2.5 Frank Robinson NCAA 2008
4.05 Russell Robinson NCAA 2008
2.25 JaJuan Smith NCAA 2008
3.7241379310344827 Lanny Smith NCAA 2008
4.264705882352941 Mark Tyndale NCAA 2008
N/A Salah Mejri N/A 2008
2.2857142857142856 Blake Griffin NCAA 2009
0.4722222222222222 Hasheem Thabeet NCAA 2009
4.228571428571429 James Harden NCAA 2009
3.864864864864865 Tyreke Evans NCAA 2009
5.333333333333333 Ricky Rubio International 2009
6.684210526315789 Jonny Flynn NCAA 2009
5.5588235294117645 Stephen Curry NCAA 2009
1.4705882352941178 Jordan Hill NCAA 2009
1.457142857142857 DeMar DeRozan NCAA 2009
2.0232558139534884 Brandon Jennings International 2009
5.0 Terrence Williams NCAA 2009
2.4594594594594597 Gerald Henderson NCAA 2009
1.0 Tyler Hansbrough NCAA 2009
3.21621621621621

5.875 Tweety Carter NCAA 2010
1.606060606060606 Richard Delk NCAA 2010
N/A Stephen Dennis NCAA 2010
0.59375 Jordan Eglseder NCAA 2010
4.580645161290323 James Florence NCAA 2010
1.032258064516129 Charles Garcia NCAA 2010
N/A Vernon Goodridge N/A 2010
3.09375 Kenny Hayes NCAA 2010
5.285714285714286 Tory Jackson NCAA 2010
N/A Dominique Johnson NCAA 2010
3.3 Tyren Johnson NCAA 2010
1.3714285714285714 Adam Koch NCAA 2010
2.0 Mac Koshwal NCAA 2010
2.257142857142857 Chris Kramer NCAA 2010
1.5 Dwight Lewis NCAA 2010
1.7894736842105263 Anthony Mason, Jr. NCAA 2010
1.8709677419354838 Tasmin Mitchell NCAA 2010
2.75 Kevin Palmer NCAA 2010
3.054054054054054 J.P. Prince NCAA 2010
4.285714285714286 Dzherom Rendl NCAA 2010
3.272727272727273 Scottie Reynolds NCAA 2010
N/A Matt Rogers NCAA 2010
N/A Billy Rush NCAA 2010
1.0 Omar Samhan NCAA 2010
4.825 Jon Scheyer NCAA 2010
0.9375 DeShawn Sims NCAA 2010
4.545454545454546 Edgar Sosa NCAA 2010
2.3225806451612905 Patrick Sullivan NCAA 2010
2.142857142857143 

...

1.4230769230769231 Michael Frazier NCAA 2015
N/A Luis Montero NCAA 2015
2.121212121212121 Bryce Dejean-Jones NCAA 2015
0.631578947368421 Duje Dukan NCAA 2015
1.4375 Stephan Hicks NCAA 2015
1.6666666666666667 Vince Hunter NCAA 2015
1.8235294117647058 Stanton Kidd NCAA 2015
2.4444444444444446 J.J. O'Brien NCAA 2015
1.9722222222222223 Jordan Sibert NCAA 2015
0.6578947368421053 Brandon Ashley NCAA 2015
2.838709677419355 J.J. Avila NCAA 2015
0.5517241379310345 Mike Cobbins NCAA 2015
3.357142857142857 Corey Hawkins NCAA 2015
0.3225806451612903 Jon Horford NCAA 2015
3.4571428571428573 Trevor Lacey NCAA 2015
0.9705882352941176 Yanick Moreira NCAA 2015
2.806451612903226 Terran Petteway NCAA 2015
1.6666666666666667 Michael Qualls NCAA 2015
0.8387096774193549 Jordan Railey NCAA 2015
1.1818181818181819 Joshua Smith NCAA 2015
4.633333333333334 Juwan Staten NCAA 2015
0.47368421052631576 Robert Upshaw NCAA 2015
3.84375 Corey Walden NCAA 2015
0.06451612903225806 Chris Walker NCAA 2015
4.78787878787878

0.5882352941176471 Isaiah Hartenstein International 2017
1.125 Damyean Dotson NCAA 2017
2.7142857142857144 Dillon Brooks NCAA 2017
3.0 Sterling Brown NCAA 2017
0.20689655172413793 Ike Anigbogu NCAA 2017
2.774193548387097 Sindarius Thornwell NCAA 2017
1.627906976744186 Vlatko Cancar International 2017
0.6206896551724138 Mathias Lessort International 2017
6.2 Monte Morris NCAA 2017
4.7727272727272725 Edmond Sumner NCAA 2017
2.9705882352941178 Kadeem Allen NCAA 2017
2.206896551724138 Alec Peters NCAA 2017
4.7105263157894735 Nigel Williams-Goss NCAA 2017
1.037037037037037 Jabari Bird NCAA 2017
0.9402985074626866 Sasha Vezenkov International 2017
3.490566037735849 Ognjen Jaramaz International 2017
1.5454545454545454 Jaron Blossomgame NCAA 2017
0.6274509803921569 Alpha Kaba International 2017
0.41935483870967744 Chris Boucher NCAA 2017
1.2 Luke Kornet NCAA 2017
2.212121212121212 Antonius Cleveland NCAA 2017
1.4102564102564104 Isaiah Hicks NCAA 2017
2.4 Ben Moore NCAA 2017
2.3793103448275863 

0.45454545454545453 Justin Tillman NCAA 2018
3.4642857142857144 Tyrius Walker NCAA 2018
1.5588235294117647 Isaiah Wilkins NCAA 2018
1.6388888888888888 Todd Withers NCAA 2018
1.09375 Drew Eubanks NCAA 2018
0.5675675675675675 Wenyen Gabriel NCAA 2018
4.764705882352941 Brandon Goodwin NCAA 2018
3.7666666666666666 Kendrick Nunn NCAA 2018
5.054054054054054 Theo Pinson NCAA 2018
2.193548387096774 Cam Reynolds NCAA 2018
1.0731707317073171 Duncan Robinson NCAA 2018
1.5757575757575757 Yuta Watanabe NCAA 2018
3.875 Kenrich Williams NCAA 2018
3.588235294117647 Joe Chealey NCAA 2018
0.7407407407407407 B.J. Johnson NCAA 2018
3.242424242424242 Allonzo Trier NCAA 2018
1.5945945945945945 Johnathan Williams NCAA 2018
2.764705882352941 Deng Adel NCAA 2018
2.5217391304347827 Rawle Alkins NCAA 2018
2.4857142857142858 Trevon Bluiett NCAA 2018
2.3142857142857145 Elijah Bryant NCAA 2018
1.5862068965517242 Marcus Derrickson NCAA 2018
1.696969696969697 Zach Lofton NCAA 2018
2.1025641025641026 Malik Newman NCAA

2.12 Lindell Wigginton NCAA 2019
2.84375 Anthony Edwards NCAA 2020
0.3333333333333333 James Wiseman NCAA 2020
6.769230769230769 LaMelo Ball International 2020
1.0 Patrick Williams NCAA 2020
2.0357142857142856 Isaac Okoro NCAA 2020
1.0714285714285714 Onyeka Okongwu NCAA 2020
5.393939393939394 Killian Hayes International 2020
2.161290322580645 Obi Toppin NCAA 2020
2.0 Deni Avdija International 2020
0.8064516129032258 Jalen Smith NCAA 2020
1.6333333333333333 Devin Vassell NCAA 2020
6.454545454545454 Tyrese Haliburton NCAA 2020
5.225806451612903 Kira Lewis, Jr. NCAA 2020
0.9285714285714286 Aaron Nesmith NCAA 2020
3.9545454545454546 Cole Anthony NCAA 2020
0.84375 Isaiah Stewart II NCAA 2020
2.8333333333333335 Aleksej Pokusevski International 2020
2.6 Josh Green NCAA 2020
2.3870967741935485 Saddiq Bey NCAA 2020
0.967741935483871 Precious Achiuwa NCAA 2020
3.193548387096774 Tyrese Maxey NCAA 2020
0.84375 Zeke Nnaji NCAA 2020
2.5416666666666665 Leandro Bolmaro International 2020
2.470588235294

0.0 Alex Antetokounmpo International 2021
1.0 Troy Baxter, Jr. NCAA 2021
0.41935483870967744 Giorgi Bezhanishvili NCAA 2021
5.25 Jordan Burns NCAA 2021
4.037037037037037 Matt Coleman III NCAA 2021
1.103448275862069 Derek Culver NCAA 2021
1.096774193548387 Aleem Ford NCAA 2021
0.96 Jay Huff NCAA 2021
4.290322580645161 DeJon Jarreau NCAA 2021
2.4193548387096775 Damien Jefferson NCAA 2021
4.473684210526316 Carlik Jones NCAA 2021
1.2380952380952381 A.J. Lawson NCAA 2021
1.7 Denzel Mahoney NCAA 2021
4.0 R.J. Nembhard NCAA 2021
N/A Daishen Nix N/A 2021
1.3548387096774193 Micah Potter NCAA 2021
4.64 Austin Reaves NCAA 2021
1.28 Olivier Sarr NCAA 2021
3.9642857142857144 Ja'Vonte Smart NCAA 2021
3.8076923076923075 Dru Smith NCAA 2021
3.121212121212121 D.J. Stewart, Jr. NCAA 2021
3.5454545454545454 Anthony Tarke NCAA 2021
1.6666666666666667 MaCio Teague NCAA 2021
1.875 Ibi Watson NCAA 2021


In [6]:
df_Past_Drafts = pd.DataFrame({"Name":Name,"Team":Team, "Position":Position,"Selection Year": Year, 
                               "Age":Age, "Nationality":Nationality, "Pick":Pick, "Height":Height, "Weight":Weight,"NIG":NIG,
                               "GP":GP, "MIN":MIN,"PTS":PTS, "FGM":FGM, "FGA":FGA, "FGP":FGP,
                               "TPM":TPM, "TPA":TPA, "TPP":TPP, "FTM":FTM, "FTA":FTA, "FTP":FTP,
                               "OFFR":OFFR, "DEFR":DEFR, "TRB":TRB, "AST":AST, "STL":STL, "BLK":BLK, "TOV":TOV})
print(df_Past_Drafts)

                   Name                                     Team Position  \
0          LeBron James  St. Vincent St. Mary High School (Ohio)        F   
1         Darko Milicic              KK Vrsac Swisslion (Serbia)       FC   
2       Carmelo Anthony                                 Syracuse        F   
3            Chris Bosh                             Georgia Tech       PF   
4           Dwyane Wade                                Marquette       SG   
...                 ...                                      ...      ...   
2297          Dru Smith                                 Missouri       SG   
2298  D.J. Stewart, Jr.                        Mississippi State       SG   
2299      Anthony Tarke                             Coppin State       PF   
2300       MaCio Teague                                   Baylor       PG   
2301         Ibi Watson                                   Dayton       SG   

      Selection Year Age    Nationality Pick  Height Weight   NIG  ...  \
0

In [7]:
df_Past_Drafts.to_csv("All_Draft_Picks.csv")
df_Past_Drafts.to_excel("All_Draft_Picks.xlsx")